In [133]:
## Import libraries
import os
import json
import pandas as pd
import traceback
from typing import Union
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from dotenv import load_dotenv

In [113]:
## Read variables from from env file
load_dotenv()  
OPENAI_KEY=os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_INDEX=os.getenv("PINECONE_INDEX_NAME")
AZURE_API_KEY1=os.getenv("AZURE_KEY1")
AZURE_API_KEY2=os.getenv("AZURE_KEY2")
AZURE_API_ENDPOINT=os.getenv("AZURE_ENDPOINT")

### Step 1: Read Source Data Files, Data Cleaning, Splitting to chunks, Convert and Store it in Vector Database


Data Source 1: Interface Functional Specification Document (PDF)

Data Source 2: Interface Data Flow (PDF)

Data Source 3: Production Support Issues & Resolutions (CSV)

Data Source 4: Interface Mapping Sheet (XLSX)

Data Source 5: Interface Architecture, Failure Modes & Error Handling Mechanism (PPTX)

In [123]:
## Folder Path for Data Source Files
SOURCE_FILES_PATH="C:\\Users\\ASHOKKUMAR KALIAPPAN\\Documents\\Ashok\\MSc_DataAnalytics\\Final_Project\\Doc\\"

In [124]:
## Function for PDF extraction
def read_data_from_pdf(FILE_PATH,EXTRACT_IMAGE_INPUT):
    loader_dataflow = PyPDFLoader(FILE_PATH, extract_images=EXTRACT_IMAGE_INPUT)
    pages_dataflow = loader_dataflow.load()
    return pages_dataflow

In [126]:
## Function for CSV data extraction
def read_data_from_csv(FILE_PATH):
    loader_csv = CSVLoader(file_path=FILE_PATH)
    data_csv = loader_csv.load()
    return data_csv

In [127]:
## Function for MS Office docs extraction
def read_data_from_msdocs(FILE_PATH):
    loader_mso = AzureAIDocumentIntelligenceLoader(
        api_endpoint=AZURE_API_ENDPOINT, api_key=AZURE_API_KEY1, file_path=FILE_PATH, api_model="prebuilt-layout"
    )
    pages_msofficedata= loader_mso.load()
    return pages_msofficedata

In [128]:
## Read Data Source 1: Interface Functional Specification Document (PDF)
extracted_data_fsd = read_data_from_pdf (SOURCE_FILES_PATH + "Engineering_Datahub_Interface_FSD.pdf",False)
extracted_data_fsd

[Document(page_content=' \n                                                           \n \n Page 1 of 19  \n \n    \n \n \n \n \n \n \n \n \nEngineering  Datahub \nInterface  \n \nFunctional Specification  Document  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ', metadata={'source': 'C:\\Users\\ASHOKKUMAR KALIAPPAN\\Documents\\Ashok\\MSc_DataAnalytics\\Final_Project\\Doc\\Engineering_Datahub_Interface_FSD.pdf', 'page': 0}),
 Document(page_content='                                                   \n Engineering  Datahub  Interface - Functional Specification Document  \n \n Page 2 of 19  \nTable of Contents    \n1. Overview  ................................ ................................ ................................ ................................ .............................  3 \n2. Objective  ................................ ................................ ................................ ...............

In [130]:
## Read Data Source 2: Interface Data Flow (PDF)
extracted_data_interfaceflow = read_data_from_pdf (SOURCE_FILES_PATH + "Engineering_Datahub_Interface_ProcessFlow.pdf",True)
print(extracted_data_interfaceflow[0].page_content)

Engineering Datahub Interface Process FlowEngineering Datahub Interface Process Flow
Engineering 
DatahubEngineering 
DatahubWindchill  PLM Windchill  PLM MiddlewareMiddlewareComplete  Change Notice 
Audit TaskChange Notice Interface 
Failure Task is Created
Change Notice AuditorChange Notice  Details**
Update Change Notice  
Details** in Engineering 
Datahub systemYesChange Notice is promoted to 
Resolved  State  and Resulting 
objects is promoted to Target 
State (Released / Obsolete)
No
Is data update 
successful?**Change Notice  Details
• Engineering Release
• Part / Assembly
• BOM Structure
• Part / Assembly Drawing
• Engineering Specification 
DocumentResend Change Notice Details 
to Target SystemLog Errors/Send Mail to 
Production Support  TeamWindchill Support Team fix 
the issue & complete 
Change Notice Interface 
Failure Task
Notify 
Change Notice AuditorEnd
Data Transformation as per Business 
Rules  and publish the data to 
Engineering Datahub system
Change Notice  Details

In [134]:
## Data Source 3: Production Support Issues & Resolutions (CSV)
extracted_data_supportresolution = read_data_from_csv (SOURCE_FILES_PATH + "Incident_Management_Steps.csv")
extracted_data_supportresolution

[Document(page_content='INC Number: INC0001\nDescription: Description of Issue: user wanted to change the description of Change Notice\nSteps involved: 1. Check if the Change Notice is present in Windchill\n2. Check the existing description of Change Notice\n3. Based on the state and approval follow below steps.\n4. If the Change Notice is in Resolved state, we need approval from Business, apart from that we can change the description directly.\n5. Change Notice > Actions > Edit \n6. Change the description as required\nWork Log: Step 1: Accept the ticket and move to In Progress \nStep 2: Check state of the Change Notice  \nStep 3: Based on the state of the Change Notice steps are processed \nStep 4:If the Change Notice is in Resolved state, we need approval from Business, apart from that we can change the description directly.\nStep 5: If the Change Notice is in resolved state then collect the information from user on why the description needs to be changed after the CN moved to resolv

In [135]:
## Read Data Source 4: Interface Mapping Sheet (XLSX)
extracted_data_interfacemapping = read_data_from_msdocs (SOURCE_FILES_PATH + "Engineering_Datahub_Interface_Mapping_Document.xlsx")
extracted_data_interfacemapping

[Document(page_content='Purpose\n===\n\nPurpose - Purpose of this document is to map attributes required for Engineering Interface from Windchill PLM system to Product Datahub system\n\nInterface\\_Mapping\n===\n\nData Element\n\nSr No\n\nSource - Windchill PLM\n\nMiddleware\n\nTarget - Engineering Datahub\n\nObject\n\nField Name\n\nField Description\n\nData Type\n\nLength\n\nMandatory (Y/N)\n\nRange values\n\nDefault Value\n\nRemarks\n\nWindchill PayLoad Tag\n\nTransformation Required\n\nTransformation Rules\n\nFunctionality\n\nField Description\n\nDatatype\n\nLength\n\nMandatory (Y/N)\n\nRange values\n\nDefault Value\n\nRemarks\n\nEngineering Release\n\n1\n\nChange Notice (CN)\n\nNumber\n\nNumber\n\nAlphanumeric\n\n10\n\nY\n\ncommon.changeNoticeNumber\n\nN\n\nEngineering Release\n\nRelease Number (Change)\n\nAlphanumeric\n\n10\n\nY\n\nEngineering Release\n\n2\n\nDefault to \'CN\'\n\nengineeringRelease.erType\n\nN\n\nEngineering Release\n\nER Type\n\nAlphabetic\n\n3\n\nY\n\nEngineerin

In [136]:
## Data Source 5: Interface Architecture, Failure Modes & Error Handling Mechanism (PPTX)
extracted_data_archdata = read_data_from_msdocs (SOURCE_FILES_PATH + "Engineering_Datahub_InterfaceArchitecture_FailureMode_ErrorHandling.pptx")
extracted_data_archdata

[Document(page_content='Engineering Datahub Interface\n===\n\nSolution Architecture, Failure Modes and Error Handling\n===\n\nInterface Solution Architecture\n===\n\nMiddleware\n\nEngineering Datahub\n\nTransform the data and publish to target system\n\nProcess and update CN details\n\nWindchill\n\nCN Audit Task Completion\n\nInterface Process Steps:\n\nWindchill will extract and publish Change Notice details to Middleware upon CN Audit Task Completion\n\nMiddleware will Transform the CN data based on transformation rules and publish to Engineering Datahub system\n\nEngineering Datahub system will process the CN details and send the processing status (Success or Failure) to Middleware\n\nMiddleware will further publish the processing status (Success or Failure) to Windchill PLM system\n\nWindchill will process the response and Release CN if the processing status is Success or create Interface Failure Task if the processing status is Failure\n\n1\n\nJSON: Change Notice details\n\n2\n\nP

### Step 2: Data Cleansing

In [57]:
## Function to clean data
def clean_pdf():
    ## convert to lowercase, lammatization, remove stop words and special chars (regex) (POS / Named Entity Recognition)
    
    return 

### Step 3: Split Data into Chunks

In [137]:
## Function to split document into chunks
def chunk_data(docs,CHUNK_SIZE,CHUNK_OVERLAP):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,chunk_overlap=CHUNK_OVERLAP)
    doc=text_splitter.split_documents(docs)
    return doc

In [147]:
## Chunk Interface FSD
documents_fsd=chunk_data(extracted_data_fsd,500,50)
len(documents_fsd)

107

In [149]:
## Chunk Interface Flow
documents_interfaceflow=chunk_data(extracted_data_interfaceflow,500,50)
len(documents_interfaceflow)

3

In [150]:
## Chunk Incident Resolution
documents_supportresolution=chunk_data(extracted_data_supportresolution,500,50)
len(documents_supportresolution)

31

In [151]:
## Chunk Interface Mapping
documents_mapping=chunk_data(extracted_data_interfacemapping,500,50)
len(documents_mapping)

15

In [152]:
## Chunk Architecture Details
documents_archdata=chunk_data(extracted_data_archdata,500,50)
len(documents_archdata)

3

### Step 4: Embeddings: Convert Chunks to Vectors and Store in Vector DB

In [142]:
## Embedding using OpenAI - model='text-embedding-ada-002'
embeddings = OpenAIEmbeddings(api_key=OPENAI_KEY)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002009622B370>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000020096233100>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [143]:
## Get the vector size / diemnsion of the embedding
vector_size = embeddings.embed_query("This is a test sentence.")

len(vector_size)

1536

In [145]:
## Pinecone VectorDB initiation, prior to this created pinecone with dimension = 1536
pc = Pinecone(api_key=PINECONE_API_KEY)
index=pc.Index(PINECONE_INDEX)

In [148]:
## Create embeddings for Interface FSD (Data Source 1)
pinecone_index = PineconeVectorStore.from_documents(documents_fsd, embeddings, index_name=PINECONE_INDEX)

In [153]:
## Create embeddings for Interface Flow Diagram (Data Source 2)
pinecone_index = PineconeVectorStore.from_documents(documents_interfaceflow, embeddings, index_name=PINECONE_INDEX)

In [154]:
## Create embeddings for Support Ticket Resolution (Data Source 3)
pinecone_index = PineconeVectorStore.from_documents(documents_supportresolution, embeddings, index_name=PINECONE_INDEX)

In [155]:
## Create embeddings for Interface Mapping (Data Source 4)
pinecone_index = PineconeVectorStore.from_documents(extracted_data_interfacemapping, embeddings, index_name=PINECONE_INDEX)

In [156]:
## Create embeddings for Interface Arch Data (Data Source 5)
pinecone_index = PineconeVectorStore.from_documents(documents_archdata, embeddings, index_name=PINECONE_INDEX)

### Step 5: RAG Implementation - Leverage Open AI Model, take input query from user & retrieve similar vectors from Vector DB, pass query + similar vectors to get the response

In [157]:
# Prompt Template

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful support assistant. Please respond to the IT application support executive queries"),
        ("user","Question:{question}")
    ]
)

In [158]:
## Cosine Similarity Retreive Results from Pinecone VectorDB
def retrieve_query(query,k=5):
    matching_results=pinecone_index.similarity_search(query,k=k)
    return matching_results

In [171]:
## OpenAI Model - gpt-3.5-turbo and chain creation
llm=ChatOpenAI(openai_api_key=OPENAI_KEY,model_name="gpt-3.5-turbo", temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")
#question_answer_chain = create_stuff_documents_chain(llm, prompt)
#rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [172]:
## Search answers for the user query
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [173]:
user_query = "What is Engineering Datahub Interface Trigger?"
answer = retrieve_answers(user_query)
print(answer)

The Engineering Datahub Interface Trigger is the Change Notice Audit Task Completion.


In [162]:
user_query = "What are the data elements extracted from Change Notice?"
answer = retrieve_answers(user_query)
print(answer)

The data elements extracted from Change Notice are:
1. Part
2. Assembly
3. Part/Assembly Drawing
4. Engineering Specification Document


In [163]:
user_query = "What are the pre-validations of Engineering Datahub Interface?"
answer = retrieve_answers(user_query)
print(answer)

The pre-validations of the Engineering Datahub Interface include CA validations and Engineering Datahub interface pre-validations.


In [164]:
user_query = "What is the middleware transforamtion rules for 'Part Maturity' attribute in Engineering Datahub Interface?"
answer = retrieve_answers(user_query)
print(answer)

I don't have specific information on the middleware transformation rules for the 'Part Maturity' attribute in the Engineering Datahub Interface based on the provided context.


In [165]:
user_query = "What are the steps involved in resolving Engineering Datahub Interface failure?"
answer = retrieve_answers(user_query)
print(answer)

The steps involved in resolving Engineering Datahub Interface failure typically include:

1. Conducting a failure analysis to identify the root cause of the interface failure.
2. Taking necessary steps to address the identified issue or error that caused the failure.
3. Re-triggering the Engineering Datahub interface to attempt to successfully transfer the data.
4. Seeking updates on the resolution from the Windchill Production Support Team.

Please note that these steps may vary depending on the specific nature of the interface failure.


In [166]:
user_query = "What are the failure modes of Engineering Datahub Interface?"
answer = retrieve_answers(user_query)
print(answer)

The failure modes of the Engineering Datahub Interface can include middleware failures, interface failures, and any other issues that prevent the successful processing of information sent by Windchill. These failures can result in the need for the Windchill Production Support Team to analyze and resolve the issue before re-triggering the Engineering Datahub interface.


In [167]:
user_query = "What are the process steps of Engineering Datahub Interface?"
answer = retrieve_answers(user_query)
print(answer)

Based on the provided information, the process steps of the Engineering Datahub Interface include:

1. Creation of a Change Notice Interface Failure Task
2. Update Change Notice Details in the Engineering Datahub system
3. Promotion of Change Notice to Resolved State
4. Promotion of Resulting objects to Target State (Released / Obsolete)
5. Extraction of data elements such as Engineering Release, Part / Assembly, BOM Structure, Part / Assembly Drawing, and Engineering Specification Document
6. Promotion of Part to Released state and extraction of data in JSON format file with Engineering Release and Item tags

These steps involve tasks related to change notices, data extraction, and promotion of objects within the Engineering Datahub Interface process flow.


### Step 6: Chat UI enablement through streamlit